# One hot encoding и train test split

In [1]:
import pandas as pd
import numpy as np


import os
import os.path as os_path

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

# current project file name
cpfn = "diplom_project"

curr_dir = os.getcwd()
curr_dir = os_path.join(curr_dir[:curr_dir.find(cpfn)], cpfn)
data_storage = os_path.join(curr_dir, "data_storage")
model_data_dir = os_path.join(data_storage, "proc_2")
fresh_data_dir = os_path.join(data_storage, "proc_old_model")
fiting_and_valid_fold = os_path.join(data_storage, "fiting_and_valid")

# Загрузка данных

In [2]:
model_data = pd.read_csv(model_data_dir + '.csv', index_col = 0)
fresh_data = pd.read_csv(fresh_data_dir + '.csv', index_col = 0)


model_Y = model_data['Y']
fresh_Y = model_data['Y']

model_X = model_data.drop('Y', axis = 1)
fresh_X = fresh_data.drop('Y', axis = 1)


# One hot encoding

In [3]:
ohe = OneHotEncoder(sparse = False).fit(
    model_X\
    .loc[:,model_X.dtypes == "O"]
)


model_X_ohe = pd.concat([
    pd.DataFrame(
        ohe.transform(model_X.loc[:,model_X.dtypes == "O"]),
        columns = ohe.get_feature_names_out(),
        index = model_X.index),
    model_X.loc[:, model_X.dtypes != "O"]],
    axis = 1)

fresh_X_ohe = pd.concat([
    pd.DataFrame(
        ohe.transform(fresh_X.loc[:,fresh_X.dtypes == "O"]),
        columns = ohe.get_feature_names_out(),
        index = fresh_X.index),
    fresh_X.loc[:, fresh_X.dtypes != 'O']],
    axis = 1)

# Normalization

In [4]:
model_X_ohe = pd.DataFrame(
    normalize(model_X_ohe.to_numpy()), columns = model_X_ohe.columns, 
    index = model_X_ohe.index)

# Train test split

In [5]:
model_X_train, model_X_test, model_Y_train, model_Y_test = train_test_split(
    model_X_ohe, model_Y, random_state = 0, stratify = model_Y)

# Сохранение результата

In [6]:
model_X_train.to_csv(os_path.join(fiting_and_valid_fold, "X_train.csv"))
model_X_test.to_csv(os_path.join(fiting_and_valid_fold, "X_test.csv"))
model_Y_train.to_csv(os_path.join(fiting_and_valid_fold, "Y_train.csv"))
model_Y_test.to_csv(os_path.join(fiting_and_valid_fold, "Y_test.csv"))

fresh_X_ohe.to_csv(os_path.join(fiting_and_valid_fold, "fresh_X.csv"))
fresh_Y.to_csv(os_path.join(fiting_and_valid_fold, "fresh_Y.csv"))